# Marketing directo con Amazon SageMaker XGBoost y ajuste de hiperparámetros

## Aprendizaje supervisado con árboles de gradiente reforzado: Un problema de predicción binaria con clases desequilibradas

### Antecedentes
El marketing directo, ya sea por correo, correo electrónico, teléfono, etc., es una táctica común para adquirir clientes. Dado que los recursos y la atención del cliente son limitados, el objetivo es dirigirse únicamente al subconjunto de clientes potenciales que probablemente se comprometan con una oferta específica. Predecir esos clientes potenciales basándose en la información disponible como los datos demográficos, las interacciones anteriores y los factores ambientales es un problema común de aprendizaje automático.

Este cuaderno entrenará un modelo que puede utilizarse para predecir si un cliente se inscribirá en un depósito a plazo en un banco, después de una o más llamadas telefónicas. Se utilizará el ajuste de hiperparámetros para probar múltiples ajustes de hiperparámetros y producir el mejor modelo.

### Preparativos
Empecemos por especificar:

* El bucket de S3 y el prefijo que desea utilizar para los datos de entrenamiento y del modelo. Esto debería estar dentro de la misma región que el entrenamiento de SageMaker.
* El rol IAM utilizado para dar acceso al entrenamiento a sus datos. Consulte la documentación de SageMaker para saber cómo crearlos.

In [1]:
import sagemaker
import boto3

import numpy as np  # For matrix operations and numerical processing
import pandas as pd  # For munging tabular data
from time import gmtime, strftime
import os

region = boto3.Session().region_name
smclient = boto3.Session().client("sagemaker")

role = sagemaker.get_execution_role()

bucket = sagemaker.Session().default_bucket()
prefix = "sagemaker/DEMO-hpo-xgboost-dm"
print(f"Bucket: {bucket}")

### Descarga de datos
Comencemos descargando el conjunto de datos de marketing directo del Repositorio ML de la UCI.

In [8]:
!wget -N https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
!unzip -o bank-additional.zip

--2022-03-10 21:58:27--  https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 444572 (434K) [application/x-httpd-php]
Saving to: ‘bank-additional.zip’

100%[======================================>] 444,572     1.39MB/s   in 0.3s   

2022-03-10 21:58:28 (1.39 MB/s) - ‘bank-additional.zip’ saved [444572/444572]

Archive:  bank-additional.zip
   creating: bank-additional/
  inflating: bank-additional/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/bank-additional/
  inflating: __MACOSX/bank-additional/._.DS_Store  
  inflating: bank-additional/.Rhistory  
  inflating: bank-additional/bank-additional-full.csv  
  inflating: bank-additional/bank-additional-names.txt  
  inflating: bank-additional/bank-additional.csv  
  inflating: __MACOSX/

In [9]:
data = pd.read_csv("./bank-additional/bank-additional-full.csv", sep=";")
pd.set_option("display.max_columns", 500)  # Make sure we can see all of the columns
pd.set_option("display.max_rows", 50)  # Keep the output on one page
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

### Sobre los datos.  

De una manera rápida, podemos ver:

* Tenemos un poco más de 40K registros de clientes, y 20 características para cada cliente
* Las características son mixtas; algunas numéricas, otras categóricas
* Los datos parecen estar ordenados, al menos por " time " y " contact ", tal vez más

Específicos en cada una de las características:**_

*Datos demográficos
* `age`: La edad del cliente (numérica)
* `job`: Tipo de trabajo (categórico: 'admin.', 'servicios', ...)
* `marital`: Estado civil (categórico: 'casado', 'soltero', ...)
* `education`: Nivel de estudios (categórico: 'básico.4y', 'bachillerato', ...)

*Eventos de clientes anteriores:*
* `default`: ¿Tiene un crédito en mora? (categórico: 'no', 'desconocido', ...)
* `housing`: ¿Tiene un crédito para la vivienda? (categórico: 'no', 'sí', ...)
* `loan`: ¿Tiene un préstamo personal? (categórico: "no", "sí", ...)

*Contactos anteriores de marketing directo:*
* `contact`: Tipo de comunicación del contacto (categórico: 'celular', 'teléfono', ...)
* `month`: Último mes del año del contacto (categórico: 'may', 'nov', ...)
* `day_of_week`: Día de la semana del último contacto (categórico: 'mon', 'fri', ...)
* `duration`: Duración del último contacto, en segundos (numérico). Nota importante: Si duración = 0 entonces `y` = 'no'.
 
*Información de la campaña
* `campaign`: Número de contactos realizados durante esta campaña y para este cliente (numérico, incluye el último contacto)
* `pdays`: Número de días transcurridos desde que el cliente fue contactado por última vez desde una campaña anterior (numérico)
* `previous`: Número de contactos realizados antes de esta campaña y para este cliente (numérico)
* `poutcome`: Resultado de la campaña de marketing anterior (categórico: "inexistente", "éxito", ...)

*Factores del entorno externo:*
* `emp.var.rate`: Tasa de variación del empleo - indicador trimestral (numérico)
* `cons.price.idx`: Índice de precios al consumo - indicador mensual (numérico)
* `cons.conf.idx`: Índice de confianza del consumidor - indicador mensual (numérico)
* `euribor3m`: Euribor 3 meses - indicador diario (numérico)
* `nr.employed`: Número de empleados - indicador trimestral (numérico) Número de empleados - indicador trimestral (numérico)

*Variable objetivo:*
* `y`: ¿Ha suscrito el cliente un depósito a plazo? (binario: "sí", "no")

## Transformación_de_datos
La limpieza de datos forma parte de casi todos los proyectos de aprendizaje automático.  Podría decirse que presenta el mayor riesgo si se hace incorrectamente y es uno de los aspectos más subjetivos del proceso.  Varias técnicas comunes incluyen:

* Manejar los valores perdidos: Algunos algoritmos de aprendizaje automático son capaces de manejar los valores perdidos, pero la mayoría no lo hacen.  Las opciones incluyen:
 * Eliminar las observaciones con valores perdidos: Esto funciona bien si sólo una fracción muy pequeña de las observaciones tiene información incompleta.
 * Eliminación de características con valores perdidos: Esto funciona bien si hay un pequeño número de características que tienen un gran número de valores perdidos.
 * Imputar los valores perdidos: Se han escrito [libros enteros](https://www.amazon.com/Flexible-Imputation-Missing-Interdisciplinary-Statistics/dp/1439868247) sobre este tema, pero las opciones más comunes son sustituir el valor que falta por la moda o la media de los valores no perdidos de esa columna.
* Conversión de valores categóricos a numéricos: El método más común es una codificación en caliente, que para cada característica asigna cada valor distinto de esa columna a su propia característica que toma un valor de 1 cuando la característica categórica es igual a ese valor, y 0 en caso contrario.
* Datos distribuidos de forma extraña: Aunque para los modelos no lineales, como los Gradient Boosted Trees, esto tiene implicaciones muy limitadas, los modelos paramétricos como la regresión pueden producir estimaciones muy inexactas cuando se alimentan con datos muy sesgados.  En algunos casos, basta con tomar el logaritmo natural de las características para obtener datos con una distribución más normal.  En otros, resulta útil dividir los valores en rangos discretos.  Estos intervalos pueden tratarse como variables categóricas e incluirse en el modelo cuando se codifica uno en caliente.
* Manejo de tipos de datos más complicados: La manipulación de imágenes, texto o datos con granos variables.

Afortunadamente, algunos de estos aspectos ya han sido manejados por nosotros, y el algoritmo que estamos mostrando tiende a hacer un buen manejo de datos dispersos o distribuidos de forma extraña.  Por lo tanto, vamos a mantener el preprocesamiento simple.

En primer lugar, muchos registros tienen el valor de "999" para *pdays*, número de días que pasaron después de que un cliente fuera contactado por última vez. Es muy probable que sea un número mágico para representar que no se hizo ningún contacto antes. Teniendo en cuenta esto, creamos una nueva columna llamada "no_previous_contact", y le damos el valor "1" cuando pdays es 999 y "0" en caso contrario.

En la columna *job*, hay categorías que significan que el cliente no está trabajando, por ejemplo, "estudiante", "jubilado" y "desempleado". Como es muy probable que el hecho de que un cliente trabaje o no afecte a su decisión de inscribirse en el depósito a plazo, generamos una nueva columna para mostrar si el cliente está trabajando basándonos en la columna *job*.

Por último, pero no por ello menos importante, convertimos los datos categóricos en numéricos, como se ha sugerido anteriormente.

In [14]:
# Variable indicadora para capturar cuando pdays toma un valor de 999
data["no_previous_contact"] = np.where(
        data["pdays"] == 999, 1, 0)

# Indicador para personas que no trabajan activamente
data["not_working"] = np.where(
        np.in1d(data["job"], ["student", "retired", "unemployed"]), 1, 0)  

# Convertir variables categóricas en conjuntos de indicadores
model_data = pd.get_dummies(data)
model_data.head()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,marital_unknown,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,default_no,default_unknown,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0


Otra pregunta que hay que hacerse antes de construir un modelo es si ciertas características añadirán valor en su caso de uso final.  Por ejemplo, si su objetivo es ofrecer la mejor predicción, ¿tendrá acceso a esos datos en el momento de la predicción?  Saber que va a llover es muy predictivo para la venta de paraguas, pero predecir el tiempo con suficiente antelación para planificar el inventario de paraguas es probablemente tan difícil como predecir la venta de paraguas sin conocer el tiempo.  Por lo tanto, incluir esto en su modelo puede darle una falsa sensación de precisión.

Siguiendo esta lógica, eliminemos las características económicas y la `duración` de nuestros datos, ya que tendrían que ser pronosticadas con gran precisión para utilizarlas como entradas en futuras predicciones.

Incluso si utilizáramos los valores de los indicadores económicos del trimestre anterior, es probable que este valor no sea tan relevante para los prospectos contactados a principios del siguiente trimestre como los contactados más tarde.

In [15]:
model_data = model_data.drop(
    ["duration", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"],
    axis=1,
)

A continuación, dividiremos el conjunto de datos en conjuntos de datos de entrenamiento (70%), de validación (20%) y de prueba (10%) y los convertiremos al formato adecuado que espera el algoritmo. Utilizaremos los conjuntos de datos de entrenamiento y validación durante el entrenamiento. El conjunto de datos de prueba se utilizará para evaluar el rendimiento del modelo una vez desplegado en un punto final.

El algoritmo **XGBoost** de *Amazon SageMaker* espera datos en formato `libSVM` o `CSV`.  Para este ejemplo, nos ceñiremos a CSV.  Ten en cuenta que la primera columna debe ser la variable de destino y que el CSV no debe incluir cabeceras.  Además, observe que, aunque sea repetitivo, es más fácil hacer esto después de la división entrenamiento-validación-prueba que hacerlo antes. Esto evita cualquier problema de desfase debido al reordenamiento aleatorio.

In [50]:
train_data, validation_data, test_data = np.split(
    model_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(model_data)), int(0.9 * len(model_data))],
)

pd.concat([train_data["y_yes"], train_data.drop(["y_no", "y_yes"], axis=1)], axis=1).to_csv(
    "train.csv", index=False, header=False
)
pd.concat(
    [validation_data["y_yes"], validation_data.drop(["y_no", "y_yes"], axis=1)], axis=1
).to_csv("validation.csv", index=False, header=False)
pd.concat([test_data["y_yes"], test_data.drop(["y_no", "y_yes"], axis=1)], axis=1).to_csv(
    "test.csv", index=False, header=False
)

Ahora copiaremos el archivo a S3 para que el entrenamiento de Amazon SageMaker lo recoja.

In [51]:
boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "train/train.csv")
).upload_file("train.csv")

boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "validation/validation.csv")
).upload_file("validation.csv")

## Configuración de Hyperparameter_Tuning 

*Nota, con la configuración por defecto de abajo, el trabajo de ajuste de hiperparámetros puede tardar unos 30 minutos en completarse.*

Ahora que hemos preparado el conjunto de datos, estamos listos para entrenar los modelos. Antes de hacerlo, hay que tener en cuenta que hay ajustes del algoritmo que se denominan "hiperparámetros" que pueden afectar drásticamente al rendimiento de los modelos entrenados. Por ejemplo, el algoritmo XGBoost tiene docenas de hiperparámetros y tenemos que elegir los valores correctos para esos hiperparámetros con el fin de lograr los resultados deseados en el entrenamiento del modelo. Dado que la configuración de los hiperparámetros puede conducir al mejor resultado también depende del conjunto de datos, es casi imposible elegir la mejor configuración de hiperparámetros sin buscarla, y un buen algoritmo de búsqueda puede buscar la mejor configuración de hiperparámetros de forma automatizada y eficaz.

Utilizaremos el ajuste de hiperparámetros de SageMaker para automatizar el proceso de búsqueda de forma efectiva. Específicamente, especificamos un rango, o una lista de posibles valores en el caso de hiperparámetros categóricos, para cada uno de los hiperparámetros que planeamos ajustar. El ajuste de hiperparámetros de SageMaker lanzará automáticamente múltiples trabajos de entrenamiento con diferentes ajustes de hiperparámetros, evaluará los resultados de esos trabajos de entrenamiento basándose en una "métrica objetiva" predefinida, y seleccionará los ajustes de hiperparámetros para futuros intentos basándose en los resultados anteriores. Para cada trabajo de ajuste de hiperparámetros, le daremos un presupuesto (número máximo de trabajos de entrenamiento) y se completará una vez que se hayan ejecutado ese número de trabajos de entrenamiento.

Ahora configuramos el trabajo de ajuste de hiperparámetros definiendo un objeto JSON que especifica la siguiente información:
* Los rangos de hiperparámetros que queremos ajustar
* Número de trabajos de entrenamiento a ejecutar en total y cuántos trabajos de entrenamiento deben ejecutarse simultáneamente. Un mayor número de trabajos paralelos terminará el ajuste antes, pero puede sacrificar la precisión. Recomendamos establecer el valor de los trabajos paralelos a menos del 10% del número total de trabajos de entrenamiento (lo estableceremos más alto sólo para este ejemplo para mantenerlo corto).
* La métrica objetiva que se utilizará para evaluar los resultados del entrenamiento, en este ejemplo, seleccionamos *validación:auc* para que sea la métrica objetiva y el objetivo es maximizar el valor a lo largo del proceso de ajuste de los hiperparámetros. Una cosa a tener en cuenta es que la métrica objetivo tiene que estar entre las métricas que son emitidas por el algoritmo durante el entrenamiento. En este ejemplo, el algoritmo XGBoost incorporado emite un montón de métricas y *validación:auc* es una de ellas. Si usted trae su propio algoritmo a SageMaker, entonces necesita asegurarse de que cualquier métrica objetiva que seleccione, su algoritmo realmente la emita.

Vamos a sintonizar cuatro hiperparámetros en estos ejemplos:
* *eta*: Reducción del tamaño del paso utilizado en las actualizaciones para evitar el sobreajuste. Después de cada paso de boosting, puede obtener directamente los pesos de las nuevas características. El parámetro eta en realidad encoge los pesos de las características para que el proceso de boosting sea más conservador. 
* *alpha*: Término de regularización L1 en los pesos. El aumento de este valor hace que los modelos sean más conservadores. 
* *min_child_weight*: Suma mínima de peso de instancia (hessian) necesaria en un hijo. Si el paso de partición del árbol da como resultado un nodo hoja con la suma del peso de la instancia menor que min_child_weight, el proceso de construcción renuncia a seguir partiendo. En los modelos de regresión lineal, esto corresponde simplemente a un número mínimo de instancias necesarias en cada nodo. Cuanto más grande sea el algoritmo, más conservador será. 
* *profundidad_máxima*: Profundidad máxima de un árbol. Aumentar este valor hace que el modelo sea más complejo y probablemente se sobreajuste. 


In [52]:
from time import gmtime, strftime, sleep

tuning_job_name = "xgboost-tuningjob-" + strftime("%d-%H-%M-%S", gmtime())

print(tuning_job_name)

tuning_job_config = {
    "ParameterRanges": {
        "CategoricalParameterRanges": [],
        "ContinuousParameterRanges": [
            {
                "MaxValue": "1",
                "MinValue": "0",
                "Name": "eta",
            },
            {
                "MaxValue": "10",
                "MinValue": "1",
                "Name": "min_child_weight",
            },
            {
                "MaxValue": "2",
                "MinValue": "0",
                "Name": "alpha",
            },
        ],
        "IntegerParameterRanges": [
            {
                "MaxValue": "10",
                "MinValue": "1",
                "Name": "max_depth",
            }
        ],
    },
    "ResourceLimits": {"MaxNumberOfTrainingJobs": 20, "MaxParallelTrainingJobs": 3},
    "Strategy": "Bayesian",
    "HyperParameterTuningJobObjective": {"MetricName": "validation:auc", "Type": "Maximize"},
}

xgboost-tuningjob-10-22-45-31


A continuación configuramos los trabajos de entrenamiento que lanzará el trabajo de ajuste de hiperparámetros definiendo un objeto JSON que especifica la siguiente información:
* La imagen contenedora del algoritmo (XGBoost)
* La configuración de entrada para los datos de entrenamiento y validación
* La configuración para la salida del algoritmo
* Los valores de los hiperparámetros del algoritmo que no se ajustan en el trabajo de ajuste (StaticHyperparameters)
* El tipo y el número de instancias a utilizar para los trabajos de entrenamiento
* La condición de parada para los trabajos de entrenamiento

Una vez más, como estamos utilizando el algoritmo XGBoost incorporado aquí, emite dos métricas predefinidas: *validación:auc* y *entrenamiento:auc*, y hemos elegido controlar *validación_auc* como puedes ver arriba. Una cosa a tener en cuenta es que si traes tu propio algoritmo, tu algoritmo emite métricas por sí mismo. En ese caso, necesitarás añadir un objeto MetricDefinition aquí para definir el formato de esas métricas a través de regex, para que SageMaker sepa cómo extraer esas métricas.

In [53]:
from sagemaker.image_uris import retrieve

training_image = retrieve(framework="xgboost", region=region, version="latest")

s3_input_train = "s3://{}/{}/train".format(bucket, prefix)
s3_input_validation = "s3://{}/{}/validation/".format(bucket, prefix)

training_job_definition = {
    "AlgorithmSpecification": {"TrainingImage": training_image, "TrainingInputMode": "File"},
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "CompressionType": "None",
            "ContentType": "csv",
            "DataSource": {
                "S3DataSource": {
                    "S3DataDistributionType": "FullyReplicated",
                    "S3DataType": "S3Prefix",
                    "S3Uri": s3_input_train,
                }
            },
        },
        {
            "ChannelName": "validation",
            "CompressionType": "None",
            "ContentType": "csv",
            "DataSource": {
                "S3DataSource": {
                    "S3DataDistributionType": "FullyReplicated",
                    "S3DataType": "S3Prefix",
                    "S3Uri": s3_input_validation,
                }
            },
        },
    ],
    "OutputDataConfig": {"S3OutputPath": "s3://{}/{}/output".format(bucket, prefix)},
    "ResourceConfig": {"InstanceCount": 1, "InstanceType": "ml.m4.xlarge", "VolumeSizeInGB": 10},
    "RoleArn": role,
    "StaticHyperParameters": {
        "eval_metric": "auc",
        "num_round": "100",
        "objective": "binary:logistic",
        "rate_drop": "0.3",
        "tweedie_variance_power": "1.4",
    },
    "StoppingCondition": {"MaxRuntimeInSeconds": 43200},
}

## Lanzar Hyperparameter_Tuning
Ahora podemos lanzar un trabajo de ajuste de hiperparámetros llamando al API create_hyper_parameter_tuning_job. Una vez creado el trabajo de ajuste de hiperparámetros, podemos ir a la consola de SageMaker para seguir el progreso del trabajo de ajuste de hiperparámetros hasta que se complete.

In [54]:
smclient.create_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name,
    HyperParameterTuningJobConfig=tuning_job_config,
    TrainingJobDefinition=training_job_definition,
)

{'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-east-2:528038902135:hyper-parameter-tuning-job/xgboost-tuningjob-10-22-45-31',
 'ResponseMetadata': {'RequestId': '04cfc609-7b36-4a5f-b47f-5d9d1b687ff5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '04cfc609-7b36-4a5f-b47f-5d9d1b687ff5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '130',
   'date': 'Thu, 10 Mar 2022 22:48:03 GMT'},
  'RetryAttempts': 0}}

Vamos a realizar una rápida comprobación del estado de los trabajos de ajuste de hiperparámetros para asegurarnos de que se han iniciado con éxito.

In [73]:
smclient.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)[
    "HyperParameterTuningJobStatus"
]

'Completed'

### Analizar los resultados del trabajo de ajuste

Una vez que haya completado un trabajo de ajuste, (o incluso mientras el trabajo todavía está en marcha) puede utilizar este cuaderno para analizar los resultados y entender cómo cada hiperparámetro afecta a la calidad del modelo.

### Seguimiento del progreso del trabajo de ajuste de hiperparámetros

Después de lanzar un trabajo de ajuste, puede ver su progreso llamando a la API describe_tuning_job. La salida de describe-tuning-job es un objeto JSON que contiene información sobre el estado actual del trabajo de ajuste. Puede llamar a list_training_jobs_for_tuning_job para ver una lista detallada de los trabajos de entrenamiento que el trabajo de ajuste lanzó.

In [67]:
# ejecuta esta celda para comprobar el estado actual del trabajo de ajuste de hiperparámetros
tuning_job_result = smclient.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)

status = tuning_job_result["HyperParameterTuningJobStatus"]
if status != "Completed":
    print("Reminder: the tuning job has not been completed.")

job_count = tuning_job_result["TrainingJobStatusCounters"]["Completed"]
print("%d training jobs have completed" % job_count)

objective = tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]
is_minimize = objective["Type"] != "Maximize"
objective_name = objective["MetricName"]

20 training jobs have completed


In [68]:
from pprint import pprint

if tuning_job_result.get("BestTrainingJob", None):
    print("Best model found so far:")
    pprint(tuning_job_result["BestTrainingJob"])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2022, 3, 10, 23, 6, 55, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:auc',
                                                 'Value': 0.7769209742546082},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2022, 3, 10, 23, 9, 51, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-2:528038902135:training-job/xgboost-tuningjob-10-22-45-31-019-f15edccb',
 'TrainingJobName': 'xgboost-tuningjob-10-22-45-31-019-f15edccb',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2022, 3, 10, 23, 8, 23, tzinfo=tzlocal()),
 'TunedHyperParameters': {'alpha': '1.5759603893020242',
                          'eta': '0.06375323166729442',
                          'max_depth': '7',
                          'min_child_weight': '9.629946028324238'}}


### Obtener todos los resultados como DataFrame
Podemos listar los hiperparámetros y las métricas objetivas de todos los trabajos de entrenamiento y elegir el trabajo de entrenamiento con la mejor métrica objetiva.

In [69]:
tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", None)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

Number of training jobs with valid objective: 20
{'lowest': 0.7198780179023743, 'highest': 0.7769209742546082}


,alpha,eta,max_depth,min_child_weight,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
1,1.575960,0.063753,7.0,9.629946,xgboost-tuningjob-10-22-45-31-019-f15edccb,Completed,0.776921,2022-03-10 23:08:23+00:00,2022-03-10 23:09:51+00:00,88.0
15,0.376414,0.106154,5.0,7.293703,xgboost-tuningjob-10-22-45-31-005-86d942b8,Completed,0.776532,2022-03-10 22:52:45+00:00,2022-03-10 22:54:08+00:00,83.0
2,0.501456,0.063591,5.0,1.840453,xgboost-tuningjob-10-22-45-31-018-84d4e4f8,Completed,0.776368,2022-03-10 23:07:35+00:00,2022-03-10 23:08:57+00:00,82.0
17,1.620037,0.395727,2.0,4.391373,xgboost-tuningjob-10-22-45-31-003-d21709ea,Completed,0.776005,2022-03-10 22:49:47+00:00,2022-03-10 22:51:04+00:00,77.0
3,0.481456,0.053591,5.0,1.750453,xgboost-tuningjob-10-22-45-31-017-798d05bb,Completed,0.775881,2022-03-10 23:05:30+00:00,2022-03-10 23:06:52+00:00,82.0
4,0.607005,0.057284,8.0,3.186640,xgboost-tuningjob-10-22-45-31-016-cf07eadb,Completed,0.775064,2022-03-10 23:05:32+00:00,2022-03-10 23:06:49+00:00,77.0
0,1.990389,0.326734,3.0,7.488161,xgboost-tuningjob-10-22-45-31-020-c891b349,Completed,0.774077,2022-03-10 23:08:34+00:00,2022-03-10 23:09:56+00:00,82.0
13,1.391146,0.234510,4.0,6.381289,xgboost-tuningjob-10-22-45-31-007-e079aa23,Completed,0.772774,2022-03-10 22:55:48+00:00,2022-03-10 22:57:10+00:00,82.0
19,0.899955,0.083153,9.0,4.056670,xgboost-tuningjob-10-22-45-31-001-2ab3453e,Completed,0.772436,2022-03-10 22:49:44+00:00,2022-03-10 22:51:06+00:00,82.0
16,1.690716,0.078748,2.0,5.664980,xgboost-tuningjob-10-22-45-31-004-f9fbaf0a,Completed,0.770894,2022-03-10 22:52:46+00:00,2022-03-10 22:53:59+00:00,73.0


### Ver resultados de *TuningJob* frente al tiempo

A continuación mostraremos cómo cambia la métrica del objetivo a lo largo del tiempo, a medida que progresa el trabajo de ajuste. En el caso de la estrategia bayesiana, debería esperar ver una tendencia general hacia mejores resultados, pero este progreso no será constante, ya que el algoritmo necesita equilibrar la exploración de nuevas áreas del espacio de parámetros con la explotación de las áreas buenas conocidas. Esto puede darle una idea de si el número de trabajos de entrenamiento es suficiente para la complejidad de su espacio de búsqueda.

In [70]:
import bokeh
import bokeh.io

bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper:
    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append((k, "@{%s}" % k))

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools="pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset"):
        return [self.hovertool(), standard_tools]

Loading BokehJS ...

In [71]:
hover = HoverHelper(tuner)
p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type="datetime")
p.circle(source=df, x="TrainingStartTime", y="FinalObjectiveValue")
show(p)

### Analizar la correlación entre la métrica objetivo y los hiperparámetros individuales
Ahora que ha terminado un trabajo de ajuste, puede querer saber la correlación entre su métrica objetivo y los hiperparámetros individuales que ha seleccionado para ajustar. Esta información le ayudará a decidir si tiene sentido ajustar los rangos de búsqueda de ciertos hiperparámetros y comenzar otro trabajo de ajuste. Por ejemplo, si ve una tendencia positiva entre la métrica del objetivo y un hiperparámetro numérico, probablemente querrá establecer un rango de ajuste más alto para ese hiperparámetro en su próximo trabajo de ajuste.

La siguiente celda dibuja un gráfico para cada hiperparámetro para mostrar su correlación con su métrica objetiva.

In [72]:
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get("Values"):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0

        vals = hp_range["Values"]
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args["x_range"] = vals

    # Now plot it
    p = figure(
        plot_width=500,
        plot_height=500,
        title="Objective vs %s" % hp_name,
        tools=hover.tools(),
        x_axis_label=hp_name,
        y_axis_label=objective_name,
        **categorical_args,
    )
    p.circle(source=df, x=hp_name, y="FinalObjectiveValue")
    figures.append(p)
show(bokeh.layouts.Column(*figures))